In [1]:
import findspark
findspark.init()

In [4]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[*]') \
        .appName('Homework') \
        .getOrCreate()

22/03/01 11:14:09 WARN Utils: Your hostname, sebastian2296 resolves to a loopback address: 127.0.1.1; using 192.168.104.122 instead (on interface wlp2s0)
22/03/01 11:14:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/01 11:14:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Q1

spark.version

'3.2.0'

In [8]:
# Q2 Size of HVFHW February 2021 
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-03-01 11:21:00--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.160.219, 2800:800:900:4::34d8:a0db
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.160.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699,83M   991KB/s    in 17m 27s 

2022-03-01 11:38:28 (685 KB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [16]:
from pyspark.sql import types

schema = types.StructType([

    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True)
]
)

In [17]:
df = spark.read.option('header', 'true').schema(schema).csv('./fhvhv_tripdata_2021-02.csv')

In [21]:
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 2g"

In [24]:
#!mkdir homework
df = df.repartition(24)

In [25]:
df.write.parquet('./homework', mode = 'overwrite')

22/03/01 11:50:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 11:50:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 11:50:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 11:50:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 11:50:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/01 11:50:26 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [31]:
!du -sh homework/

209M	homework/


## Read Parquet Files

In [33]:
dfp = spark.read.parquet('./homework/*')

In [36]:
dfp.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [38]:
# Q3 How many taxi trips were there on February 15?
dfp.createOrReplaceTempView('fhvh')

In [41]:
spark.sql('''
    SELECT COUNT(1) as trips_15 FROM fhvh
    WHERE EXTRACT(DAY from pickup_datetime) = 15 
''').show()

+--------+
|trips_15|
+--------+
|  367170|
+--------+



In [50]:
# Q4 Trip starting on which day was the longest?
spark.sql('''
    SELECT pickup_datetime, EXTRACT(HOUR FROM (dropoff_datetime - pickup_datetime))  as trip_duration FROM fhvh
    ORDER BY 2 DESC
''').show()

+-------------------+-------------+
|    pickup_datetime|trip_duration|
+-------------------+-------------+
|2021-02-11 13:40:44|           20|
|2021-02-17 15:54:53|           15|
|2021-02-20 12:08:15|           12|
|2021-02-03 20:24:25|           11|
|2021-02-19 23:17:44|           10|
|2021-02-18 15:24:19|            9|
|2021-02-25 09:18:18|            9|
|2021-02-10 01:56:17|            9|
|2021-02-18 01:31:20|            9|
|2021-02-10 20:51:39|            9|
|2021-02-25 17:13:35|            9|
|2021-02-20 01:36:13|            9|
|2021-02-21 22:50:52|            8|
|2021-02-09 13:30:13|            8|
|2021-02-21 19:59:13|            8|
|2021-02-08 12:04:24|            8|
|2021-02-02 09:42:30|            8|
|2021-02-03 11:18:31|            8|
|2021-02-10 10:12:08|            8|
|2021-02-12 06:16:42|            8|
+-------------------+-------------+
only showing top 20 rows



In [55]:
# Q5 Now find the most frequently occurring dispatching_base_num in this dataset. 
# How many stages this spark job has?

spark.sql('''
    SELECT  dispatching_base_num, COUNT(dispatching_base_num) FROM fhvh
    GROUP BY 1
    ORDER BY 2 DESC

''').show()


+--------------------+---------------------------+
|dispatching_base_num|count(dispatching_base_num)|
+--------------------+---------------------------+
|              B02510|                    3233664|
|              B02764|                     965568|
|              B02872|                     882689|
|              B02875|                     685390|
|              B02765|                     559768|
|              B02869|                     429720|
|              B02887|                     322331|
|              B02871|                     312364|
|              B02864|                     311603|
|              B02866|                     311089|
|              B02878|                     305185|
|              B02682|                     303255|
|              B02617|                     274510|
|              B02883|                     251617|
|              B02884|                     244963|
|              B02882|                     232173|
|              B02876|         

In [62]:
# Q6 Find the most common pickup-dropoff pair.

In [63]:
zones = spark.read.parquet('./zones/*')
zones.createOrReplaceTempView('zones')

In [98]:
fhvh_zones = spark.sql('''
    SELECT CONCAT(zpu.Zone, '/ ', zdo.Zone) as MCL, COUNT(zdo.Zone) FROM fhvh f JOIN zones zpu ON f.PULocationID = zpu.LocationID
    JOIN zones zdo ON f.DOLocationID = zdo.LocationID
    GROUP BY 1
    ORDER BY 2 DESC
''').show(truncate=False)

+----------------------------------------------------+-----------+
|MCL                                                 |count(Zone)|
+----------------------------------------------------+-----------+
|East New York/ East New York                        |45041      |
|Borough Park/ Borough Park                          |37329      |
|Canarsie/ Canarsie                                  |28026      |
|Crown Heights North/ Crown Heights North            |25976      |
|Bay Ridge/ Bay Ridge                                |17934      |
|Jackson Heights/ Jackson Heights                    |14688      |
|Astoria/ Astoria                                    |14688      |
|Central Harlem North/ Central Harlem North          |14481      |
|Bushwick South/ Bushwick South                      |14424      |
|Flatbush/Ditmas Park/ Flatbush/Ditmas Park          |13976      |
|South Ozone Park/ South Ozone Park                  |13716      |
|Brownsville/ Brownsville                            |12829   